# Homework 3: Supervised machine learning

UIC CS 418, Spring 2020

_According to the **Academic Integrity Policy** of this course, all work submitted for grading must be done individually, unless otherwise specified. While we encourage you to talk to your peers and learn from them, this interaction must be superficial with regards to all work submitted for grading. This means you cannot work in teams, you cannot work side-by-side, you cannot submit someone else’s work (partial or complete) as your own. In particular, note that you are guilty of academic dishonesty if you extend or receive any kind of unauthorized assistance. Absolutely no transfer of program code between students is permitted (paper or electronic), and you may not solicit code from family, friends, or online forums. Other examples of academic dishonesty include emailing your program to another student, copying-pasting code from the internet, working in a group on a homework assignment, and allowing a tutor, TA, or another individual to write an answer for you. Academic dishonesty is unacceptable, and penalties range from failure to expulsion from the university; cases are handled via the official student conduct process described at https://dos.uic.edu/conductforstudents.shtml._

This homework is an individual assignment for all graduate students. Undergraduate students are allowed to work in pairs and submit one homework assignment per pair. There will be no extra credit given to undergraduate students who choose to work alone. The pairs of students who choose to work together and submit one homework assignment together still need to abide by the Academic Integrity Policy and not share or receive help from others (except each other).


## Due Date

This assignment is due at 11:59pm Friday, March 13, 2020. 


### What to Submit

You need to complete all code and answer all questions denoted by **Q#** (each one is under a bike image) in this notebook. When you are done, you should export **`hw3.ipynb`** with your answers as a PDF file, upload that file `hw3.pdf` to *Homework 3 - Written Part* on Gradescope, tagging each question. 

You need to copy all functions that are part of questions Q1-Q9 to `hw3.py` (a template has been provided). That includes `process()`, `process_all()`, `create_features()`, `create_labels()`, `class MajorityLabelClassifier()`, `learn_classifier()`, `evaluate_classifier()`, `best_model_selection()` and `classify_tweets()`. You need to upload a completed Jupyter notebook (`hw3.ipynb` file) and `hw3.py` to *Homework 3 - code* on Gradescope. 

For undergraduate students who work in a team of two, only one student needs to submit the homework and just tag the other student on Gradescope.

#### Autograding

Questions will be graded based on both manual grading and an Autograder which will run on your `hw3.py` file. All parts of Homework 3 are graded based on correctness, **not** based on completion.

In [31]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import sklearn 
import string
from IPython.display import display, Latex, Markdown

# Classifying tweets [100%]

In this problem, you will be analyzing Twitter data extracted using [this](https://dev.twitter.com/overview/api) api. The data contains tweets posted by the following six Twitter accounts: `realDonaldTrump, mike_pence, GOP, HillaryClinton, timkaine, TheDemocrats`

For every tweet, there are two pieces of information:
- `screen_name`: the Twitter handle of the user tweeting and
- `text`: the content of the tweet.

The tweets have been divided into two parts - train and test available to you in CSV files. For train, both the `screen_name` and `text` attributes were provided but for test, `screen_name` is hidden.

The overarching goal of the problem is to "predict" the political inclination (Republican/Democratic) of the Twitter user from one of his/her tweets. The ground truth (i.e., true class labels) is determined from the `screen_name` of the tweet as follows
- `realDonaldTrump, mike_pence, GOP` are Republicans
- `HillaryClinton, timkaine, TheDemocrats` are Democrats

Thus, this is a binary classification problem. 

The problem proceeds in three stages:
- **Text processing (25%)**: We will clean up the raw tweet text using the various functions offered by the [nltk](http://www.nltk.org/genindex.html) package.
- **Feature construction (25%)**: In this part, we will construct bag-of-words feature vectors and training labels from the processed text of tweets and the `screen_name` columns respectively.
- **Classification (50%)**: Using the features derived, we will use [sklearn](http://scikit-learn.org/stable/modules/classes.html) package to learn a model which classifies the tweets as desired. 

You will use two new python packages in this problem: `nltk` and `sklearn`, both of which should be available with anaconda. However, NLTK comes with many corpora, toy grammars, trained models, etc, which have to be downloaded manually. This assignment requires NLTK's stopwords list and WordNetLemmatizer. Install them using:

In [32]:
nltk.download('stopwords')
nltk.download('wordnet')

# Verify that the following commands work for you, before moving on.

lemmatizer=nltk.stem.wordnet.WordNetLemmatizer()
stopwords=nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Elizabeth\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Elizabeth\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Let's begin!

## A. Text Processing [25%]

You first task to fill in the following function which processes and tokenizes raw text. The generated list of tokens should meet the following specifications:
1. The tokens must all be in lower case.
2. The tokens should appear in the same order as in the raw text.
3. The tokens must be in their lemmatized form. If a word cannot be lemmatized (i.e, you get an exception), simply catch it and ignore it. These words will not appear in the token list.
4. The tokens must not contain any punctuations. Punctuations should be handled as follows: (a) Apostrophe of the form `'s` must be ignored. e.g., `She's` becomes `she`. (b) Other apostrophes should be omitted. e.g, `don't` becomes `dont`. (c) Words must be broken at the hyphen and other punctuations. 
5. The tokens must not contain any part of a url.

Part of your work is to figure out a logical order to carry out the above operations. You may find `string.punctuation` useful, to get hold of all punctuation symbols. Look for [regular expressions](https://docs.python.org/3/library/re.html) capturing urls in the text. Your tokens must be of type `str`. Use `nltk.word_tokenize()` for tokenization once you have handled punctuation in the manner specified above. 

You would want to take a look at the `lemmatize()` function [here](https://www.nltk.org/_modules/nltk/stem/wordnet.html).
In order for `lemmatize()` to give you the root form for any word, you have to provide the context in which you want to lemmatize through the `pos` parameter: `lemmatizer.lemmatize(word, pos=SOMEVALUE)`. The context should be the part of speech (POS) for that word. The good news is you do not have to manually write out the lexical categories for each word because [nltk.pos_tag()](https://www.nltk.org/book/ch05.html) will do this for you. Now you just need to use the results from `pos_tag()` for the `pos` parameter.
However, you can notice the POS tag returned from `pos_tag()` is in different format than the expected pos by `lemmatizer`.
> pos
(Syntactic category): n for noun files, v for verb files, a for adjective files, r for adverb files.

You need to map these pos appropriately. `nltk.help.upenn_tagset()` provides description of each tag returned by `pos_tag()`.

<img src="bikeshare.png" width="100px" align="left" float="left"/>
<br><br><br>

## Q1 (15%):

In [33]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import re

def process(text, lemmatizer=nltk.stem.wordnet.WordNetLemmatizer()):
    """ Normalizes case and handles punctuation
    Inputs:
        text: str: raw text
        lemmatizer: an instance of a class implementing the lemmatize() method
                    (the default argument is of type nltk.stem.wordnet.WordNetLemmatizer)
    Outputs:
        list(str): tokenized text
    """
    
    #Remove appropiate apostrophes ('s) from string
    tempText = ''
    apos = 0
    for t in text:
        if (t == '\'' or t == '’'):
            apos = 1
        else:
            if (apos == 1 and t == 's'):
                apos = 0
                continue
            else:
                apos = 0
            tempText += t
            
    #Remove valid urls        
    pattern = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    all_urls = re.findall(pattern, tempText)
    for url in all_urls:
        if (url[len(url) - 1] == ')'):
            url = url[:-1]
        newText = ''
        if(re.search(url, tempText)):
            begin, end = re.search(url, tempText).span()
            for i in range(len(tempText)):
                if (i >= begin and i < end): continue
                else: newText += tempText[i]
            tempText = newText
            
    #Break words at punctuations/remove punctuations
    finalText = ''
    for char in tempText:
        if (char in string.punctuation or char == '—' or char == '–'):
            char = ' '
        finalText += char
    
    #Tokenize string
    final_list = nltk.word_tokenize(finalText.lower())

    #Pos tagging and lemmatzing
    pos = nltk.pos_tag(final_list)
    return_list = []
    for t in pos:
        if(t[1][0] == 'V'): p = 'v'
        elif(t[1][0] == 'J'): p = 'a'
        elif(t[1][0] == 'R'): p = 'r'
        else: p = 'n'
        temp = lemmatizer.lemmatize(t[0], p)
        return_list.append(temp)

    return return_list

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Elizabeth\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Elizabeth\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


You can test the above function as follows. Try to make your test strings as exhaustive as possible.

In [34]:
#print(process("Mr.Kaine's doing well! How about you?"))
# ['im', 'do', 'well', 'how', 'about', 'you']

#print(process("Education is the ability to listen to almost anything without losing your temper or your self-confidence."))
# ['education', 'be', 'the', 'ability', 'to', 'listen', 'to', 'almost', 'anything', 'without', 'lose', 'your', 'temper', 'or', 'your', 'self', 'confidence']

#print(process("Have you committed to vote yet? Election Day is only 5 days away! http://t.co/p1n1OQzkS6 #laelex #lagov #lalege http://t.co/vd…"))

#print(process("It's hilarious. Check it out http://t.co/dummyurl"))
# ['it', 'hilarious', 'check', 'it', 'out']

#print(process("RT if you'll be watching his last #SOTU tomorrow night! (RSVP here: https://t.co/c07e92jl52) https://t.co/ZtBk92FOre"))
#print("\n")
#print(process("Thanks to you, victory is in sight. Pitch in $1, and let's go win this nomination: https://t.co/CuTsRc8HKK"))
#print("\n")
#print(process("RT @DWStweets: Saal Mubarak to the millions of Americans who celebrated Diwali this week! https://t.co/f2UifKn2aV"))
#print("\n")
#print(process("RT @GOPconvention: #FridayFact When did the very first @GOPconvention take place?"))
#print("\n")
#print(process("\"We've got to make this economy work for everyone, not just those at the top.\" —Hillary: https://t.co/Et5D2WbDNI"))
#print("\n")
#print(process("We ultimately want to invest $1 bill over the next 4yrs to improve what is already a good system of roads &amp; bridges."))
#print("\n")
#print(process("“I am here to tell you that Hillary Clinton is as tough as they come.” –Martin O’Malley #DemsInPhilly https://t.co/hmOuM8FjNi"))
#print("\n")
#print(process("Agree w/@HillaryClinton's call for no-fly zone in Syria. Aids effort to address worst humanitarian crisis since WWII https://t.co/X1hvQEPdfF"))
#print("\n")
#print(process("Foreign policy expert Bremmer: \"\"Our country is not in decline, but our foreign policy sure the hell is.” http://t.co/md9nDiFvXu"))
#print("\n")
print(process("\"Some issues are not left vs. right or liberal vs. conservative, they are right vs. wrong.\" —Rev. William Barber https://t.co/LRZRS9Qmqu"))
print("\n")
print(process("IAEA inspections = key to detecting any Iranian nuclear activity (and avoiding mistakes of past) cc: @POLITICOMag https://t.co/alV2jFiN2O"))
#print(process("RT @realDonaldTrump: HAPPY 241st BIRTHDAY to the @USArmy! THANK YOU! https://t.co/mXsxkfcstC"))

['some', 'issue', 'be', 'not', 'leave', 'v', 'right', 'or', 'liberal', 'v', 'conservative', 'they', 'be', 'right', 'v', 'wrong', 'rev', 'william', 'barber']


['iaea', 'inspection', 'key', 'to', 'detect', 'any', 'iranian', 'nuclear', 'activity', 'and', 'avoid', 'mistake', 'of', 'past', 'cc', 'politicomag']


<img src="bikeshare.png" width="100px" align="left" float="left"/>
<br><br><br>

## Q2 (10%):

You will now use the `process()` function we implemented to convert the pandas dataframe we just loaded from tweets_train.csv file. Your function should be able to handle any data frame which contains a column called `text`. The data frame you return should replace every string in `text` with the result of `process()` and retain all other columns as such. Do not change the order of rows/columns. Before writing `process_all()`, load the data into a DataFrame and look at its format:

In [35]:
tweets = pd.read_csv("tweets_train.csv", na_filter=False)
print(tweets.head())

      screen_name                                               text
0             GOP  RT @GOPconvention: #Oregon votes today. That m...
1    TheDemocrats  RT @DWStweets: The choice for 2016 is clear: W...
2  HillaryClinton  Trump's calling for trillion dollar tax cuts f...
3  HillaryClinton  .@TimKaine's guiding principle: the belief tha...
4        timkaine  Glad the Senate could pass a #THUD / MilCon / ...


In [36]:
def process_all(df, lemmatizer=nltk.stem.wordnet.WordNetLemmatizer()):
    """ process all text in the dataframe using process() function.
    Inputs
        df: pd.DataFrame: dataframe containing a column 'text' loaded from the CSV file
        lemmatizer: an instance of a class implementing the lemmatize() method
                    (the default argument is of type nltk.stem.wordnet.WordNetLemmatizer)
    Outputs
        pd.DataFrame: dataframe in which the values of text column have been changed from str to list(str),
                        the output from process() function. Other columns are unaffected.
    """
    df["text"] = df["text"].apply(lambda row: process(row))
    return df

In [37]:
# test your code
processed_tweets = process_all(tweets)
print(processed_tweets.head())

#       screen_name                                               text
# 0             GOP  [rt, gopconvention, oregon, vote, today, that,...
# 1    TheDemocrats  [rt, dwstweets, the, choice, for, 2016, be, cl...
# 2  HillaryClinton  [trump, call, for, trillion, dollar, tax, cut,...
# 3  HillaryClinton  [timkaine, guide, principle, the, belief, that...
# 4        timkaine  [glad, the, senate, could, pass, a, thud, milc...

      screen_name                                               text
0             GOP  [rt, gopconvention, oregon, vote, today, that,...
1    TheDemocrats  [rt, dwstweets, the, choice, for, 2016, be, cl...
2  HillaryClinton  [trump, call, for, trillion, dollar, tax, cut,...
3  HillaryClinton  [timkaine, guide, principle, the, belief, that...
4        timkaine  [glad, the, senate, could, pass, a, thud, milc...


## B. Feature Construction [25%]

The next step is to derive feature vectors from the tokenized tweets. In this section, you will be constructing a bag-of-words TF-IDF feature vector. But before that, as you may have guessed, the number of possible words is prohibitively large and not all of them may be useful for our classification task. We need to determine which words to retain, and which to omit. A common heuristic is to construct a frequency distribution of words in the corpus and prune out the head and tail of the distribution. The intuition of the above operation is as follows. Very common words (i.e. stopwords) add almost no information regarding similarity of two pieces of text. Similarly with very rare words. NLTK has a list of in-built stop words which is a good substitute for head of the distribution. We will consider a word rare if it occurs only in a single document (row) in whole of `tweets_train.csv`. 

<img src="bikeshare.png" width="100px" align="left" float="left"/>
<br><br><br>

## Q3 (15%):

Construct a sparse matrix of features for each tweet with the help of `sklearn.feature_extraction.text.TfidfVectorizer` (documentation [here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)). You need to pass a parameter `min_df=2` to filter out the words occuring only in one document in the whole training set. Remember to ignore the stop words as well. You must leave other optional parameters (e.g., `vocab`, `norm`, etc) at their default values. 

In [38]:
def create_features(processed_tweets, stop_words):
    """ creates the feature matrix using the processed tweet text
    Inputs:
        tweets: pd.DataFrame: tweets read from train/test csv file, containing the column 'text'
        stop_words: list(str): stop_words by nltk stopwords
    Outputs:
        sklearn.feature_extraction.text.TfidfVectorizer: the TfidfVectorizer object used
            we need this to tranform test tweets in the same way as train tweets
        scipy.sparse.csr.csr_matrix: sparse bag-of-words TF-IDF feature matrix
    """
    
    vector = sklearn.feature_extraction.text.TfidfVectorizer(stop_words=stop_words, min_df=2)
    array = []
    for t in processed_tweets["text"]:
        string = ''
        for word in t:
            string += word
            string += ' '
        array.append(string)
    sparse_matrix = vector.fit_transform(array)
    return vector, sparse_matrix

#create_features(processed_tweets, stopwords)

In [39]:
# execute this code 
(tfidf, X) = create_features(processed_tweets, stopwords)

<img src="bikeshare.png" width="100px" align="left" float="left"/>
<br><br><br>

## Q4 (10%):

Also for each tweet, assign a class label (0 or 1) using its `screen_name`. Use 0 for realDonaldTrump, mike_pence, GOP and 1 for the rest.

In [40]:
def create_labels(processed_tweets):
    """ creates the class labels from screen_name
    Inputs:
        tweets: pd.DataFrame: tweets read from train file, containing the column 'screen_name'
    Outputs:
        numpy.ndarray(int): dense binary numpy array of class labels
    """
    array = np.zeros(len(processed_tweets["screen_name"]))
    curr = processed_tweets["screen_name"].apply(lambda row: process(row))
    counter = 0
    for c in curr:
        if(c[0] == "gop" or c[0] == "mike" or c[0] == "realdonaldtrump"): array[counter] = 0
        else: array[counter] = 1
        counter += 1
    return array

In [41]:
# execute this code
y = create_labels(processed_tweets)

## C. Classification [50%]

And finally, we are ready to put things together and learn a model for the classification of tweets. The classifier you will be using is [`sklearn.svm.SVC`](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC) (Support Vector Machine). 

At the heart of SVMs is the concept of kernel functions, which determines how the similarity/distance between two data points in computed. `sklearn`'s SVM provides four kernel functions: `linear`, `poly`, `rbf`, `sigmoid` (details [here](http://scikit-learn.org/stable/modules/svm.html#svm-kernels)) but you can also implement your own distance function and pass it as an argument to the classifier.

Through the various functions you implement in this part, you will be able to learn a classifier, score a classifier based on how well it performs, use it for prediction tasks and compare it to a baseline.

Specifically, you will carry out the following tasks (Q5-9) in order:

1. Implement and evaluate a simple baseline classifier MajorityLabelClassifier.
2. Implement the `learn_classifier()` function assuming `kernel` is always one of {`linear`, `poly`, `rbf`, `sigmoid`}. 
3. Implement the `evaluate_classifier()` function which scores a classifier based on accuracy of a given dataset.
4. Implement `best_model_selection()` to perform cross-validation by calling `learn_classifier()` and `evaluate_classifier()` for different folds and determine which of the four kernels performs the best.
5. Go back to `learn_classifier()` and fill in the best kernel. 

<img src="bikeshare.png" width="100px" align="left" float="left"/>
<br><br><br>

## Q5 (10%):

To determine whether your classifier is performing well, you need to compare it to a baseline classifier. A baseline is generally a simple or trivial classifier and your classifier should beat the baseline in terms of a performance measure such as accuracy. Implement a classifier called `MajorityLabelClassifier` that always predicts the class equal to **mode** of the labels (i.e., the most frequent label) in training data. Part of the code is done for you. Implement the `fit` and `predict` methods. Initialize your classifier appropriately.

In [42]:
class MajorityLabelClassifier():
  """
  A classifier that predicts the mode of training labels
  """
  def __init__(self):
    """
    Initialize
    """
    self.mode = None
  
  def fit(self, X, y):
    """
    Implement fit by taking training data X and their labels y and finding the mode of y
    """
    count0 = 0
    count1 = 0
    for i in y:
        if(i == 0): count0 += 1
        else: count1 += 1
    if(count1 > count0):
        self.mode = 1
    else: self.mode = 0
    return self

  def predict(self, X):
    """
    Implement to give the mode of training labels as a prediction for each data instance in X
    return labels
    """
    array = np.zeros(X.shape[0])
    for i in range(X.shape[0]):
        array[i] = self.mode
    return array

# Report the accuracy of your classifier by comparing the predicted label of each example to its true label
classifier = MajorityLabelClassifier()
classifier.fit(X, y)
classifier.predict(X)
# training accuracy = 0.500173

array([1., 1., 1., ..., 1., 1., 1.])

<img src="bikeshare.png" width="100px" align="left" float="left"/>
<br><br><br>

## Q6 (10%):

Implement the `learn_classifier()` function assuming `kernel` is always one of {`linear`, `poly`, `rbf`, `sigmoid`}. Stick to default values for any other optional parameters.

In [43]:
def learn_classifier(X_train, y_train, kernel):
    """ learns a classifier from the input features and labels using the kernel function supplied
    Inputs:
        X_train: scipy.sparse.csr.csr_matrix: sparse matrix of features, output of create_features()
        y_train: numpy.ndarray(int): dense binary vector of class labels, output of create_labels()
        kernel: str: kernel function to be used with classifier. [linear|poly|rbf|sigmoid]
    Outputs:
        sklearn.svm.classes.SVC: classifier learnt from data
    """
    
    svc = sklearn.svm.classes.SVC(kernel=kernel)
    svc.fit(X_train, y_train)
    return svc

In [44]:
# execute code
classifier = learn_classifier(X, y, 'linear')
print(classifier)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)


<img src="bikeshare.png" width="100px" align="left" float="left"/>
<br><br><br>

## Q7 (10%):

Now that we know how to learn a classifier, the next step is to evaluate it, ie., characterize how good its classification performance is. This step is necessary to select the best model among a given set of models, or even tune hyperparameters for a given model.

There are two questions that should now come to your mind:
1. **What data to use?** 
    - **Validation Data**: The data used to evaluate a classifier is called **validation data** (or hold-out data), and it is usually different from the data used for training. The model or hyperparameter with the best performance in the held out data is chosen. This approach is relatively fast and simple but vulnerable to biases found in validation set.
    - **Cross-validation**: This approach divides the dataset in $k$ groups (so, called k-fold cross-validation). One of group is used as test set for evaluation and other groups as training set. The model or hyperparameter with the best average performance across all k folds is chosen. For this question you will perform 4-fold cross validation to determine the best kernel. We will keep all other hyperparameters default for now. This approach provides robustness toward biasness in validation set. However, it takes more time.
    
2. **And what metric?** There are several evaluation measures available in the literature (e.g., accuracy, precision, recall, F-1,etc) and different fields have different preferences for specific metrics due to different goals. We will go with accuracy. According to wiki, **accuracy** of a classifier measures the fraction of all data points that are correctly classified by it; it is the ratio of the number of correct classifications to the total number of (correct and incorrect) classifications. `sklearn.metrics` provides a number of performance metrics.

Now, implement the following function.

In [45]:
def evaluate_classifier(classifier, X_validation, y_validation):
    """ evaluates a classifier based on a supplied validation data
    Inputs:
        classifier: sklearn.svm.classes.SVC: classifer to evaluate
        X_train: scipy.sparse.csr.csr_matrix: sparse matrix of features
        y_train: numpy.ndarray(int): dense binary vector of class labels
    Outputs:
        double: accuracy of classifier on the validation data
    """
    acc = sklearn.metrics.accuracy_score(classifier.predict(X_validation), y_validation)
    return acc

In [46]:
# test your code by evaluating the accuracy on the training data
accuracy = evaluate_classifier(classifier, X, y)
print(accuracy) 
# should give 0.956700196554515

0.951670713377269


<img src="bikeshare.png" width="100px" align="left" float="left"/>
<br><br><br>

## Q8 (10%):

Now it is time to decide which kernel works best by using the cross-validation technique. Write code to split the training data into 4-folds (75% training and 25% validation) by shuffling randomly. For each kernel, record the average accuracy for all folds and determine the best classifier. Since our dataset is balanced (both classes are in almost equal propertion), `sklearn.model_selection.KFold` [doc](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) can be used for cross-validation.

In [47]:
kf = sklearn.model_selection.KFold(n_splits=4, random_state=1, shuffle=True)
kf

KFold(n_splits=4, random_state=1, shuffle=True)

Then use the following code to determine which classifier is the best. 

In [48]:
def best_model_selection(kf, X, y):
  """
  Select the kernel giving best results using k-fold cross-validation.
  Other parameters should be left default.
  Input:
    kf (sklearn.model_selection.KFold): kf object defined above
    X (scipy.sparse.csr.csr_matrix): training data
    y (array(int)): training labels
  Return:
    best_kernel (string)
  """
  best = 0
  for kernel in ['linear', 'rbf', 'poly', 'sigmoid']:
    total = 0
    count = 0
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        classif = learn_classifier(X_train, y_train, kernel)
        acc = evaluate_classifier(classif, X_test, y_test)
        total += acc
        count += 1
    avg = total/count
    if avg > best:
        best = avg
        str = kernel
      # Use the documentation of KFold cross-validation to split ..
      # training data and test data from create_features() and create_labels()
      # call learn_classifer() using training split of kth fold
      # evaluate on the test split of kth fold
      # record avg accuracies and determine best model (kernel)
  #return best kernel as string
  return str


#Test your code
best_kernel = best_model_selection(kf, X, y)
best_kernel

C:\Users\Elizabeth\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Elizabeth\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Elizabeth\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Elizabeth\Anaconda3\lib\site-packages\sklearn\svm\base.py:

'linear'

<img src="bikeshare.png" width="100px" align="left" float="left"/>
<br><br>

## Q9 (10%)

We're almost done! It's time to write a nice little wrapper function that will use our model to classify unlabeled tweets from tweets_test.csv file. 

In [49]:
def classify_tweets(tfidf, classifier, unlabeled_tweets):
    """ predicts class labels for raw tweet text
    Inputs:
        tfidf: sklearn.feature_extraction.text.TfidfVectorizer: the TfidfVectorizer object used on training data
        classifier: sklearn.svm.classes.SVC: classifier learnt
        unlabeled_tweets: pd.DataFrame: tweets read from tweets_test.csv
    Outputs:
        numpy.ndarray(int): dense binary vector of class labels for unlabeled tweets
    """
    proc_tweets = process_all(unlabeled_tweets)
    array = []
    for t in proc_tweets["text"]:
        string = ''
        for word in t:
            string += word
            string += ' '
        array.append(string)
    sparse_matrix = tfidf.transform(array)
    val = classifier.predict(sparse_matrix)
    return val

In [50]:
# Fill in best classifier in your function and re-trian your classifier using all training data
# Get predictions for unlabelled test data
classifier = learn_classifier(X, y, best_kernel)
unlabeled_tweets = pd.read_csv("tweets_test.csv", na_filter=False)
y_pred = classify_tweets(tfidf, classifier, unlabeled_tweets)
# Printing labels for each processed tweet
for i in range(len(y_pred)):
    if y_pred[i] == 0:
        print("Republican")
    else:
        print("Democrat")
    for j in range(len(unlabeled_tweets.loc[i])):
        print(unlabeled_tweets.loc[i][j])
    print('\n')

Democrat
['a', 'comprehensive', 'look', 'at', 'the', 'many', 'lie', 'and', 'offense', 'of', 'donald', 'trump']


Democrat
['horrible', 'killing', 'of', 'a', '13', 'year', 'old', 'american', 'girl', 'at', 'her', 'home', 'in', 'israel', 'by', 'a', 'palestinian', 'terrorist', 'we', 'must', 'get', 'tough']


Democrat
['im', 'here', 'a', 'a', 'proud', 'american', 'a', 'proud', 'democrat', 'a', 'proud', 'mother', 'and', 'tonight', 'in', 'particular', 'a', 'very', 'very', 'proud', 'daughter', '”', 'chelseaclinton']


Democrat
['the', 'republican', 'candidate', 'on', 'make', 'the', 'economy', 'work', 'for', 'middle', 'class', 'american', 'gopdebate']


Republican
['great', 'way', 'to', 'start', 'my', 'day', 'skyped', 'with', 'madison', 'wetsel', 'middle', 'school', 'to', 'discuss', 'current', 'event', 'cc', 'resnickteaches']


Democrat
['hint', 'it', 'isnt', 'hillary', 'help', 'elect', 'a', 'gop', 'president', 'who', 'ready', 'to', 'lead']


Republican
['anne', 'amp', 'i', 'have', 'a', 'great'

['arena', 'be', 'pack', 'totally', 'electric']


Democrat
['imagine', 'donald', 'trump', 'sit', 'in', 'the', 'situation', 'room', 'make', 'life', 'or', 'death', 'decision', 'on', 'behalf', 'of', 'the', 'united', 'state']


Republican
['apparently', 'this', 'be', 'controversial']


Democrat
['rt', 'dwstweets', 'waybackwednesday', 'floridian', 'deserve', 'a', 'senator', 'who', 'show', 'up', 'for', 'work']


Republican
['look', 'like', 'the', 'bernie', 'people', 'will', 'fight', 'if', 'not', 'their', 'blood', 'sweat', 'and', 'tear', 'be', 'a', 'total', 'waste', 'of', 'time', 'kaine', 'stand', 'for', 'opposite']


Democrat
['iran', 'be', 'go', 'to', 'buy', '114', 'jetliner', 'with', 'a', 'small', 'part', 'of', 'the', '150', 'billion', 'we', 'be', 'give', 'them', 'but', 'they', 'wont', 'buy', 'from', 'u', 's', 'rather', 'airbus']


Democrat
['rt', 'madeleine', 'timkaine', 'that', 'what', 'we', 'wellesley', 'girl', 'do', 'and', 'it', 'so', 'clear', 'hillaryclinton', 'be', 'the', 'only', 'can